In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64

import time
from datetime import datetime


# Scikit-Learn Model Deployment Use Case

In this case, we will build a very large ensemble model (here, Random Foreast with 512 trees) on a digits dataset  (not very original !!!) and generate a SQL code for deployment using the web service. 

We then execute the SQL code on a local database (postgresql) and compare the SQL execution result with scikit-learn predict/predict_proba/.predict_log_proba result. 

Both results are stored in pandas dataframes. 


## Build a scikit-learn model

In [2]:
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
n_classes = len(digits.target_names)

In [3]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=512, max_depth=7, min_samples_leaf=30, random_state = 1960)
clf.fit(digits.data, digits.target)

RandomForestClassifier(max_depth=7, min_samples_leaf=30, n_estimators=512,
                       random_state=1960)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:


def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"duckdb"}
    r = requests.post(WS_URL, json=data)
    #print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
with open('data/digits_64_features_RF_512_DuckDB.sql', 'w') as f:
    f.write(lSQL)
N = len(lSQL)
P = 4000
print(lSQL[0:P] + "..." + lSQL[N//2:(N//2 + P)] + "..." + lSQL[-P:])

WITH "RF_0" AS 
(WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_43" <= 2.5) THEN CASE WHEN ("ADS"."Feature_26" <= 3.5) THEN CASE WHEN ("ADS"."Feature_19" <= 6.5) THEN CASE WHEN ("ADS"."Feature_28" <= 15.5) THEN 4 ELSE 5 END ELSE 6 END ELSE CASE WHEN ("ADS"."Feature_22" <= 0.5) THEN CASE WHEN ("ADS"."Feature_54" <= 2.5) THEN CASE WHEN ("ADS"."Feature_36" <= 7.5) THEN 10 ELSE 11 END ELSE CASE WHEN ("ADS"."Feature_33" <= 0.5) THEN 13 ELSE 14 END END ELSE CASE WHEN ("ADS"."Feature_34" <= 7.5) THEN CASE WHEN ("ADS"."Feature_44" <= 1.5) THEN 17 ELSE 18 END ELSE CASE WHEN ("ADS"."Feature_35" <= 2.5) THEN CASE WHEN ("ADS"."Feature_37" <= 8.5) THEN 21 ELSE 22 END ELSE 23 END END END END ELSE CASE WHEN ("ADS"."Feature_54" <= 1.5) THEN CASE WHEN ("ADS"."Feature_38" <= 0.5) THEN CASE WHEN ("ADS"."Feature_20" <= 13.5) THEN CASE WHEN ("ADS"."Feature_61" <= 0.5) THEN CASE WHEN ("ADS"."Feature_20" <= 0.5) THEN 29 ELSE 30 END ELSE 31 END ELSE CASE WHEN ("ADS"."Feature

## Execute the SQL Code

In [7]:
# save the dataset in a database table

conf = {"memory_limit": "1000mb", "temp_directory" : "/tmp"}
engine = sa.create_engine("duckdb:///:memory:" , connect_args={"config": conf}, echo=False)
conn = engine.connect()

lTable = pd.DataFrame(digits.data);
lTable.columns = ['Feature_' + str(c) for c in range(digits.data.shape[1])]
lTable['KEY'] = range(lTable.shape[0])
lTable.to_csv("data/digits.csv", index=False)
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)

print("DUCKDB_VERSION", [x for x in conn.execute("PRAGMA version")])

DUCKDB_VERSION [('0.5.1', '7c111322d')]


In [ ]:
sql_output = pd.read_sql(lSQL , conn);
sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)
conn.close()

In [ ]:
sql_output.sample(12, random_state=1960)

In [ ]:
sql_output.Decision.value_counts()

## Scikit-learn Prediction

In [ ]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_' + str(c) for c in range(n_classes)]);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_' + str(c) for c in range(n_classes)])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_' + str(c) for c in range(n_classes)])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)


## Comparing the SQL and Scikit-learn Predictions

In [ ]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [ ]:
sql_skl_join.sample(12, random_state=1960)

In [ ]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]
